In [ ]:
# !pip install --proxy http://10.208.164.185:9999 pyspark
# !pip install --proxy http://10.208.164.185:9999 -U ipykernel # để bên ngoài connect vào trong dc notebook (từ vscode)

# RUN pip3 install --proxy http://10.60.117.103:8085 jupyterlab
# pip install jupyterlab
# nohup python3 -m jupyterlab --ip=0.0.0.0 --port=18080 --NotebookApp.token=1111 --allow-root > /dev/null &

# cài pyspark mới nhất lỗi gì ấy quên rồi hic (hình như là scala )
# RUN pip3 install --proxy http://10.60.117.103:8085 pyspark==3.3.1


     |████████▏                       | 80.8 MB 194 kB/s eta 0:20:16^C

ERROR: Operation cancelled by user


In [ ]:
ghi vao mysql cua datahub 

In [ ]:
from pyspark.sql import SparkSession

config_datahub = {'spark.extraListeners': 'datahub.spark.DatahubSparkListener',
                  '\"spark.datahub.rest.server': 'http://10.208.164.167:8080\"',
                  'spark.datahub.metadata.dataset.materialize': 'true',  # hien hdfs
                  '\"spark.datahub.file_partition_regexp': '/partition=.*|/_delta_log.*\"',
                  'spark.datahub.metadata.dataset.experimental_include_schema_metadata': 'true'}   # ten cot, thogn tin cot

# một task spark sẽ đi kèm với 1 pipeline, tức là có 10job thì sẽ có 20task spark và 10 pipeline
# Tạo SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("WriteToPhoenix") \
    .config("spark.jars", "/tmp/phoenix5-spark3-6.0.0.7.2.17.0-334.jar,/tmp/phoenix-hbase-compat-2.4.1-5.1.3.jar,\
    /usr/local/phoenix/phoenix-client-hbase-2.4-5.1.3.jar,/tmp/phoenix-core-5.1.3.jar,/tmp/acryl-spark-lineage-5206f9d-SNAPSHOT.jar") \
        .config("spark.extraListeners", "datahub.spark.DatahubSparkListener")\
        .config("spark.datahub.rest.server", "http://10.208.164.167:8080")\
    .getOrCreate()

# Tạo DataFrame mẫu
data = [
    (1, "John Doe", 25),
    (2, "Jane Smith", 30),
    (3, "Alice Johnson", 22)
]
columns = ["id", "name", "age"]
df = spark.createDataFrame(data, columns)


df.show()

In [ ]:
# Cấu hình kết nối Phoenix
phoenix_url = "jdbc:phoenix:localhost"  # Thay đổi 'localhost' thành địa chỉ ZooKeeper của bạn
table_name = "USERS"

# Ghi DataFrame vào Phoenix
df.write \
    .format("org.apache.phoenix.spark") \
    .mode("overwrite") \
    .option("zkUrl", phoenix_url) \
    .option("table", table_name) \
    .save()

print("Data written to Phoenix successfully!")

# Dừng SparkSession
spark.stop()

In [ ]:
from pyspark.sql import SparkSession

# RUN pip3 install --proxy http://10.60.117.103:8085 pyspark==3.3.1 -> spak 3.5 van dang loi

# Initialize Spark session with HBase configuration
spark = SparkSession.builder \
    .appName("WriteToHBase") \
    .config("spark.jars", "/tmp/hbase-spark-1.0.1-SNAPSHOT_spark331_hbase2415.jar,\
    /tmp/hbase-spark-protocol-shaded-1.0.1-SNAPSHOT_spark331_hbase2415.jar,\
    /tmp/hbase-shaded-mapreduce-2.4.15.jar,/hbase-site.xml.jar,/tmp/scala-parser-combinators_2.12-2.1.1.jar,/tmp/htrace-core4-4.2.0-incubating.jar") \
    .getOrCreate()

    # .config("spark.hadoop.hbase.zookeeper.quorum", "localhost") \
    # .config("spark.hadoop.hbase.zookeeper.property.clientPort", "2181") \
    # .config("spark.hadoop.hbase.master", "localhost:16000") \
    # .config("spark.executor.extraClassPath", "/usr/local/hbase/lib/")\
    # .config("spark.driver.extraClassPath", "/usr/local/hbase/lib/")\
# /hbase-site.xml.jar, ?????
#     /tmp/hbase-client-2.4.14.jar,/tmp/hbase-server-2.4.14.jar,/tmp/hbase-mapreduce-2.4.15.jar
#     /tmp/hbase-spark3-protocol-shaded-1.0.0.7.2.17.0-334.jar
#    /tmp/phoenix5-spark3-6.0.0.7.2.17.0-334.jar,/tmp/phoenix-hbase-compat-2.4.1-5.1.3.jar,/tmp/scala-parser-combinators_2.12-2.1.1.jar,\
#    /usr/local/phoenix/phoenix-client-hbase-2.4-5.1.3.jar,/tmp/phoenix-core-5.1.3.jar,\
    # /tmp/hbase-client-2.4.14.jar,/tmp/hbase-server-2.4.14.jar,/tmp/hbase-mapreduce-2.4.15.jar, \